In [79]:
import psycopg2
import pandas as pd
import numpy as np 
from apyori import apriori


In [20]:
def connection(user,pwd,host,port,db):
    try:
    
       connection = psycopg2.connect(user=user,
                                  password=pwd,
                                  host=host,
                                  port=port,
                                  database=db)
       cursor = connection.cursor()
       return cursor
#    postgreSQL_select_Query = "select * from snp_db limit 5"

#    cursor.execute(postgreSQL_select_Query)
#    print("Selecting rows from snp_db table using cursor.fetchall")
#    records = cursor.fetchall() t
   
#    print("Print each row and it's columns values")
#    for row in records:
#        print(row)

    except (Exception, psycopg2.Error) as error :
        print ("Error while fetching data from PostgreSQL", error)

#     finally:
#         #closing database connection.
#         if(connection):
#             cursor.close()
#             connection.close()
#             print("PostgreSQL connection is closed")

In [19]:
user ="postgres"
pwd = "1997"
host ="localhost"
port = "5432"
db= "postgres"



PostgreSQL connection is closed


In [182]:
query = "select initialAA,finalAA from snp_db "
def select(q):
    c = connection(user,pwd,host,port,db)
    c.execute(q)
    df = pd.DataFrame(c.fetchall())

    return df


In [183]:
snp =select(query)

In [184]:
snp.columns=[ 'initialAA', 'finalAA']

In [185]:
snp

,initialAA,finalAA
0,His,Arg
1,His,Arg
2,Val,Met
3,Ala,Ser
4,Gly,Arg
...,...,...
78239,Thr,Pro
78240,Arg,His
78241,Arg,Ser
78242,Val,Leu


In [186]:
snps = []
for row in range(0,len(snp)):
    snps.append([str(snp.values[row,j]) for j in range(0,2)])

In [209]:
association_rules = apriori(snps,min_support=0.0093,min_confidence=0.250,min_lift =3,min_length=2)
association_results =list(association_rules)
pd.DataFrame(association_results)

,items,support,ordered_statistics
0,"(Asp, Asn)",0.020756,"[((Asn), (Asp), 0.28361858190709044, 3.3978643..."
1,"(Tyr, Cys)",0.021931,"[((Cys), (Tyr), 0.25475059382422804, 4.9436273..."
2,"(Glu, Lys)",0.031555,"[((Glu), (Lys), 0.36944486009277266, 5.0633812..."


In [210]:
association_results

[RelationRecord(items=frozenset({'Asp', 'Asn'}), support=0.020755585092786667, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Asn'}), items_add=frozenset({'Asp'}), confidence=0.28361858190709044, lift=3.3978643887212345)]),
 RelationRecord(items=frozenset({'Tyr', 'Cys'}), support=0.02193139410050611, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cys'}), items_add=frozenset({'Tyr'}), confidence=0.25475059382422804, lift=4.943627347019568), OrderedStatistic(items_base=frozenset({'Tyr'}), items_add=frozenset({'Cys'}), confidence=0.42559523809523814, lift=4.943627347019569)]),
 RelationRecord(items=frozenset({'Glu', 'Lys'}), support=0.031555135218035886, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Glu'}), items_add=frozenset({'Lys'}), confidence=0.36944486009277266, lift=5.06338126346101), OrderedStatistic(items_base=frozenset({'Lys'}), items_add=frozenset({'Glu'}), confidence=0.4324750394114556, lift=5.06338126346101)])]

In [221]:
finals = []
for item in association_results:                                          
#     to display the rule, the support, the confidence, and lift for each rule in a more clear way:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    finals.append((items[0],items[1]))

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: Asp -> Asn
Support: 0.020755585092786667
Confidence: 0.28361858190709044
Lift: 3.3978643887212345
Rule: Tyr -> Cys
Support: 0.02193139410050611
Confidence: 0.25475059382422804
Lift: 4.943627347019568
Rule: Glu -> Lys
Support: 0.031555135218035886
Confidence: 0.36944486009277266
Lift: 5.06338126346101


# Let's try to display the "Type_of_Variant"


In [223]:
finals #this is the association rules that we obtained from the database

[('Asp', 'Asn'), ('Tyr', 'Cys'), ('Glu', 'Lys')]

### Here we will try to display the database given initial and final amino acids are as obtained

In [265]:
final_Queries = ["select type_of_variant, count(main_gene_name) from snp_db where initialaa='Asp' and finalaa = 'Asn' group by type_of_variant;",
                "select type_of_variant, count(main_gene_name) from snp_db where initialaa='Tyr' and finalaa = 'Cys' group by type_of_variant; ",
                 "select type_of_variant, count(main_gene_name) from snp_db where initialaa='Glu' and finalaa = 'Lys' group by type_of_variant;"]
def select(q):
    c = connection(user,pwd,host,port,db)
    c.execute(q)
    df = pd.DataFrame(c.fetchall())
    return df

In [270]:
df1 = pd.DataFrame(select(final_Queries[0]))
df1.columns =['Type_of_Variant', 'Counts']
df2 = pd.DataFrame(select(final_Queries[1]))
df2.columns =['Type_of_Variant', 'Counts']
df3 = pd.DataFrame(select(final_Queries[2]))
df3.columns =['Type_of_Variant', 'Counts']

In [271]:
df1
#When  AA changes from Asp to Asn

,Type_of_Variant,Counts
0,Disease,457
1,Polymorphism,592
2,Unclassified,147


In [272]:
df2
#When  AA changes from Tyr to Lys

,Type_of_Variant,Counts
0,Disease,574
1,Polymorphism,285
2,Unclassified,80


In [273]:
df3
#When  AA changes from Glu to Asn

,Type_of_Variant,Counts
0,Disease,785
1,Polymorphism,813
2,Unclassified,214
